In [4]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

class YargitayKararlariChunker:
    def __init__(self, chunk_size=1200, chunk_overlap=150):
        # Hukuki metinler için optimize edilmiş ayırıcılar
        self.separators = [
            "\n\n",  # Paragraf arası
            ". ",    # Cümle sonu
            ", ",    # Virgül
            " ",     # Boşluk
            ""       # Karakter düzeyi
        ]
        
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=self.separators,
            length_function=len,
            is_separator_regex=False
        )
    
    def preprocess_text(self, raw_text):
        """Metni temizle ve normalize et"""
        # Fazla boşlukları temizle
        text = re.sub(r'\s+', ' ', raw_text.strip())
        
        # Özel karakterleri düzelt
        text = text.replace('""', '"')
        text = text.replace('...', '[...]')
        
        return text
    
    def extract_metadata_from_text(self, text):
        """Metinden önemli bilgileri çıkar"""
        metadata = {}
        
        # Daire bilgisi
        daire_match = re.search(r'(\d+\.\s*\w+\s*Dairesi)', text)
        if daire_match:
            metadata['daire'] = daire_match.group(1)
        
        # Mahkeme bilgisi
        mahkeme_match = re.search(r'MAHKEMESİ\s*:\s*([^\\n]+)', text)
        if mahkeme_match:
            metadata['mahkeme'] = mahkeme_match.group(1).strip()
        
        # Karar türü
        if 'reddine karar' in text.lower():
            metadata['karar_turu'] = 'RED'
        elif 'kabulune karar' in text.lower():
            metadata['karar_turu'] = 'KABUL'
        else:
            metadata['karar_turu'] = 'DİĞER'
            
        return metadata
    
    def chunk_single_karar(self, row):
        """Tek bir kararı chunk'lara böl"""
        # Temel bilgileri hazırla
        base_metadata = {
            '_id': row['_id'],
            'location': row['location'],
            'esasNo': row['esasNo'],
            'kararNo': row['kararNo'],
            'extractedDates': row['extractedDates'],
            'esasNo_num': row['esasNo_num'],
            'esasNo_tip': row['esasNo_tip'],
            'kararNo_num': row['kararNo_num'],
            'kararNo_tip': row['kararNo_tip']
        }
        
        # Metni temizle
        cleaned_text = self.preprocess_text(row['rawText'])
        
        # Metinden ek bilgi çıkar
        text_metadata = self.extract_metadata_from_text(cleaned_text)
        
        # Chunk'lara böl
        chunks = self.text_splitter.split_text(cleaned_text)
        
        # Her chunk için metadata ekle
        chunked_data = []
        for i, chunk in enumerate(chunks):
            chunk_metadata = base_metadata.copy()
            chunk_metadata.update(text_metadata)
            chunk_metadata.update({
                'chunk_id': f"{row['_id']}_chunk_{i+1}",
                'chunk_index': i + 1,
                'total_chunks': len(chunks),
                'chunk_text': chunk,
                'chunk_length': len(chunk)
            })
            chunked_data.append(chunk_metadata)
        
        return chunked_data
    
    def chunk_dataframe(self, df):
        """Tüm DataFrame'i chunk'lara böl"""
        all_chunks = []
        
        for index, row in df.iterrows():
            try:
                chunks = self.chunk_single_karar(row)
                all_chunks.extend(chunks)
                print(f"Karar {index+1}: {len(chunks)} chunk oluşturuldu")
            except Exception as e:
                print(f"Karar {index+1} işlenirken hata: {e}")
                continue
        
        return pd.DataFrame(all_chunks)

# Kullanım örneği
# CSV ile çalışma fonksiyonları
def process_csv(input_csv_path, output_csv_path, chunk_size=2000, chunk_overlap=100):
    """CSV dosyasını okur, chunk'lar ve yeni CSV dosyasına kaydeder"""
    
    # CSV'yi oku
    print(f"CSV dosyası okunuyor: {input_csv_path}")
    df = pd.read_csv(input_csv_path, encoding='utf-8')
    
    print(f"Toplam karar sayısı: {len(df)}")
    print(f"Kolonlar: {list(df.columns)}")
    
    # Chunker'ı başlat
    chunker = YargitayKararlariChunker(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    
    # Chunk'lara böl
    print("\nChunk işlemi başlıyor...")
    chunked_df = chunker.chunk_dataframe(df)
    
    # Özet bilgi
    print(f"\n=== ÖZET ===")
    print(f"İşlenen karar sayısı: {len(df)}")
    print(f"Üretilen chunk sayısı: {len(chunked_df)}")
    print(f"Ortalama chunk/karar: {len(chunked_df)/len(df):.1f}")
    
    # Chunk uzunluk istatistikleri
    chunk_lengths = chunked_df['chunk_length']
    print(f"Chunk uzunlukları - Min: {chunk_lengths.min()}, Max: {chunk_lengths.max()}, Ort: {chunk_lengths.mean():.0f}")
    
    # CSV olarak kaydet
    print(f"\nSonuç kaydediliyor: {output_csv_path}")
    chunked_df.to_csv(output_csv_path, index=False, encoding='utf-8')
    
    print(f"✅ İşlem tamamlandı!")
    print(f"📂 Giriş dosyası: {input_csv_path}")
    print(f"📄 Çıkış dosyası: {output_csv_path}")
    return chunked_df

def analyze_chunks(csv_path):
    """Chunk'lanmış CSV'yi analiz eder"""
    df = pd.read_csv(csv_path, encoding='utf-8')
    
    print(f"=== CHUNK ANALİZİ ===")
    print(f"Toplam chunk sayısı: {len(df)}")
    print(f"Benzersiz karar sayısı: {df['_id'].nunique()}")
    
    # Karar başına chunk dağılımı
    chunks_per_karar = df.groupby('_id')['chunk_index'].max()
    print(f"\nKarar başına chunk dağılımı:")
    print(f"Min: {chunks_per_karar.min()}")
    print(f"Max: {chunks_per_karar.max()}")
    print(f"Ortalama: {chunks_per_karar.mean():.1f}")
    
    # En çok chunk'a sahip kararlar
    print(f"\nEn çok chunk'a sahip 5 karar:")
    top_chunked = chunks_per_karar.nlargest(5)
    for karar_id, chunk_count in top_chunked.items():
        esasNo = df[df['_id'] == karar_id]['esasNo'].iloc[0]
        print(f"- {esasNo}: {chunk_count} chunk")

def main():
    """Örnek kullanım - Giriş ve çıkış CSV dosya yollarını belirtin"""
    
    # Dosya yollarını belirtin
    input_file = "/home/yapayzeka/ahsen_bulbul/data/10data.csv"      # Giriş CSV dosyası
    output_file = "yargitay_chunks.csv"        # Çıkış CSV dosyası
    
    try:
        # CSV'yi işle - Her zaman yeni dosyaya yazar
        chunked_data = process_csv(
            input_csv_path=input_file,
            output_csv_path=output_file,    # Zorunlu parametre
            chunk_size=1200,                # İhtiyaca göre ayarlayın
            chunk_overlap=100               # İhtiyaca göre ayarlayın
        )
        
        # Analiz yap
        analyze_chunks(output_file)
        
        return chunked_data
        
    except FileNotFoundError:
        print(f"❌ Dosya bulunamadı: {input_file}")
        print("Lütfen doğru dosya yolunu 'input_file' değişkenine girin.")
        return None
    except Exception as e:
        print(f"❌ Hata oluştu: {e}")
        return None

# Alternatif kullanım fonksiyonları
def chunk_csv_simple(input_path, output_path):
    """Basit kullanım - varsayılan ayarlarla"""
    return process_csv(input_path, output_path)

def chunk_csv_custom(input_path, output_path, chunk_size, chunk_overlap):
    """Özel ayarlarla chunk'lama"""
    return process_csv(input_path, output_path, chunk_size, chunk_overlap)

if __name__ == "__main__":
    result = main()

CSV dosyası okunuyor: /home/yapayzeka/ahsen_bulbul/data/10data.csv
Toplam karar sayısı: 20
Kolonlar: ['_id', 'location', 'rawText', 'extractedDates', 'esasNo', 'kararNo', 'esasNo_num', 'esasNo_tip', 'kararNo_num', 'kararNo_tip']

Chunk işlemi başlıyor...
Karar 1: 1 chunk oluşturuldu
Karar 2: 13 chunk oluşturuldu
Karar 3: 10 chunk oluşturuldu
Karar 4: 6 chunk oluşturuldu
Karar 5: 18 chunk oluşturuldu
Karar 6: 1 chunk oluşturuldu
Karar 7: 1 chunk oluşturuldu
Karar 8: 7 chunk oluşturuldu
Karar 9: 1 chunk oluşturuldu
Karar 10: 7 chunk oluşturuldu
Karar 11: 6 chunk oluşturuldu
Karar 12: 12 chunk oluşturuldu
Karar 13: 6 chunk oluşturuldu
Karar 14: 1 chunk oluşturuldu
Karar 15: 11 chunk oluşturuldu
Karar 16: 4 chunk oluşturuldu
Karar 17: 10 chunk oluşturuldu
Karar 18: 2 chunk oluşturuldu
Karar 19: 15 chunk oluşturuldu
Karar 20: 5 chunk oluşturuldu

=== ÖZET ===
İşlenen karar sayısı: 20
Üretilen chunk sayısı: 137
Ortalama chunk/karar: 6.8
Chunk uzunlukları - Min: 4, Max: 1200, Ort: 897

Sonuç 

###DENEME


In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

class YargitayKararlariChunker:
    def __init__(self, chunk_size=2000, chunk_overlap=100, aggressive_mode=False):
        # Aggressive mode için daha az ayırıcı kullan
        if aggressive_mode:
            self.separators = [
                "\n\n",  # Sadece paragraf arası
                ". ",    # Cümle sonu
                " "      # Boşluk
            ]
        else:
            # Hukuki metinler için optimize edilmiş ayırıcılar
            self.separators = [
                "\n\n",  # Paragraf arası
                ". ",    # Cümle sonu
                ", ",    # Virgül
                " ",     # Boşluk
                ""       # Karakter düzeyi
            ]
        
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=self.separators,
            length_function=len,
            is_separator_regex=False
        )
        
        print(f"Chunker ayarları: size={chunk_size}, overlap={chunk_overlap}, aggressive={aggressive_mode}")
    
    def preprocess_text(self, raw_text):
        """Metni temizle ve normalize et"""
        # Fazla boşlukları temizle
        text = re.sub(r'\s+', ' ', raw_text.strip())
        
        # Özel karakterleri düzelt
        text = text.replace('""', '"')
        text = text.replace('...', '[...]')
        
        return text
    
    def extract_metadata_from_text(self, text):
        """Metinden önemli bilgileri çıkar"""
        metadata = {}
        
        # Daire bilgisi
        daire_match = re.search(r'(\d+\.\s*\w+\s*Dairesi)', text)
        if daire_match:
            metadata['daire'] = daire_match.group(1)
        
        # Mahkeme bilgisi
        mahkeme_match = re.search(r'MAHKEMESİ\s*:\s*([^\\n]+)', text)
        if mahkeme_match:
            metadata['mahkeme'] = mahkeme_match.group(1).strip()
        
        # Karar türü
        if 'reddine karar' in text.lower():
            metadata['karar_turu'] = 'RED'
        elif 'kabulune karar' in text.lower():
            metadata['karar_turu'] = 'KABUL'
        else:
            metadata['karar_turu'] = 'DİĞER'
            
        return metadata
    
    def chunk_single_karar(self, row):
        """Tek bir kararı chunk'lara böl"""
        # Temel bilgileri hazırla
        base_metadata = {
            '_id': row['_id'],
            'location': row['location'],
            'esasNo': row['esasNo'],
            'kararNo': row['kararNo'],
            'extractedDates': row['extractedDates'],
            'esasNo_num': row['esasNo_num'],
            'esasNo_tip': row['esasNo_tip'],
            'kararNo_num': row['kararNo_num'],
            'kararNo_tip': row['kararNo_tip']
        }
        
        # Metni temizle
        cleaned_text = self.preprocess_text(row['rawText'])
        
        # Metinden ek bilgi çıkar
        text_metadata = self.extract_metadata_from_text(cleaned_text)
        
        # Chunk'lara böl
        chunks = self.text_splitter.split_text(cleaned_text)
        
        # Her chunk için metadata ekle
        chunked_data = []
        for i, chunk in enumerate(chunks):
            chunk_metadata = base_metadata.copy()
            chunk_metadata.update(text_metadata)
            chunk_metadata.update({
                'chunk_id': f"{row['_id']}_chunk_{i+1}",
                'chunk_index': i + 1,
                'total_chunks': len(chunks),
                'chunk_text': chunk,
                'chunk_length': len(chunk)
            })
            chunked_data.append(chunk_metadata)
        
        return chunked_data
    
    def chunk_dataframe(self, df):
        """Tüm DataFrame'i chunk'lara böl"""
        all_chunks = []
        
        for index, row in df.iterrows():
            try:
                chunks = self.chunk_single_karar(row)
                all_chunks.extend(chunks)
                print(f"Karar {index+1}: {len(chunks)} chunk oluşturuldu")
            except Exception as e:
                print(f"Karar {index+1} işlenirken hata: {e}")
                continue
        
        return pd.DataFrame(all_chunks)

# Kullanım örneği
# CSV ile çalışma fonksiyonları
def process_csv(input_csv_path, output_csv_path, chunk_size=2000, chunk_overlap=100, aggressive_mode=False):
    """CSV dosyasını okur, chunk'lar ve yeni CSV dosyasına kaydeder"""
    
    # CSV'yi oku
    print(f"CSV dosyası okunuyor: {input_csv_path}")
    df = pd.read_csv(input_csv_path, encoding='utf-8')
    
    print(f"Toplam karar sayısı: {len(df)}")
    print(f"Kolonlar: {list(df.columns)}")
    
    # Chunker'ı başlat
    chunker = YargitayKararlariChunker(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap,
        aggressive_mode=aggressive_mode
    )
    
    # Chunk'lara böl
    print("\nChunk işlemi başlıyor...")
    chunked_df = chunker.chunk_dataframe(df)
    
    # Özet bilgi
    print(f"\n=== ÖZET ===")
    print(f"İşlenen karar sayısı: {len(df)}")
    print(f"Üretilen chunk sayısı: {len(chunked_df)}")
    print(f"Ortalama chunk/karar: {len(chunked_df)/len(df):.1f}")
    print(f"Önceki chunk sayısından azalma: %{((138-len(chunked_df))/138)*100:.1f}")
    
    # Chunk uzunluk istatistikleri
    chunk_lengths = chunked_df['chunk_length']
    print(f"Chunk uzunlukları - Min: {chunk_lengths.min()}, Max: {chunk_lengths.max()}, Ort: {chunk_lengths.mean():.0f}")
    
    # CSV olarak kaydet
    print(f"\nSonuç kaydediliyor: {output_csv_path}")
    chunked_df.to_csv(output_csv_path, index=False, encoding='utf-8')
    
    print(f"✅ İşlem tamamlandı!")
    print(f"📂 Giriş dosyası: {input_csv_path}")
    print(f"📄 Çıkış dosyası: {output_csv_path}")
    return chunked_df

def analyze_chunks(csv_path):
    """Chunk'lanmış CSV'yi analiz eder"""
    df = pd.read_csv(csv_path, encoding='utf-8')
    
    print(f"=== CHUNK ANALİZİ ===")
    print(f"Toplam chunk sayısı: {len(df)}")
    print(f"Benzersiz karar sayısı: {df['_id'].nunique()}")
    
    # Karar başına chunk dağılımı
    chunks_per_karar = df.groupby('_id')['chunk_index'].max()
    print(f"\nKarar başına chunk dağılımı:")
    print(f"Min: {chunks_per_karar.min()}")
    print(f"Max: {chunks_per_karar.max()}")
    print(f"Ortalama: {chunks_per_karar.mean():.1f}")
    
    # En çok chunk'a sahip kararlar
    print(f"\nEn çok chunk'a sahip 5 karar:")
    top_chunked = chunks_per_karar.nlargest(5)
    for karar_id, chunk_count in top_chunked.items():
        esasNo = df[df['_id'] == karar_id]['esasNo'].iloc[0]
        print(f"- {esasNo}: {chunk_count} chunk")

def main():
    """Örnek kullanım - Giriş ve çıkış CSV dosya yollarını belirtin"""
    
    # Dosya yollarını belirtin
    input_file = "yargitay_kararlari.csv"      # Giriş CSV dosyası
    output_file = "yargitay_chunks.csv"        # Çıkış CSV dosyası
    
    try:
        # CSV'yi işle - Az chunk için optimize edilmiş ayarlar
        chunked_data = process_csv(
            input_csv_path=input_file,
            output_csv_path=output_file,
            chunk_size=2000,            # Büyük chunk (az chunk için)
            chunk_overlap=100,          # Az overlap
            aggressive_mode=True        # Agresif mod
        )
        
        # Analiz yap
        analyze_chunks(output_file)
        
        return chunked_data
        
    except FileNotFoundError:
        print(f"❌ Dosya bulunamadı: {input_file}")
        print("Lütfen doğru dosya yolunu 'input_file' değişkenine girin.")
        return None
    except Exception as e:
        print(f"❌ Hata oluştu: {e}")
        return None

# Chunk sayısını minimize etmek için özel fonksiyonlar
def minimize_chunks(input_path, output_path):
    """Minimum chunk sayısı için optimize edilmiş"""
    return process_csv(
        input_path, 
        output_path, 
        chunk_size=3000,        # Çok büyük chunk
        chunk_overlap=50,       # Minimum overlap
        aggressive_mode=True
    )

def balanced_chunks(input_path, output_path):
    """Dengeli yaklaşım"""
    return process_csv(
        input_path, 
        output_path, 
        chunk_size=2000,        # Orta büyüklük
        chunk_overlap=100,      # Az overlap
        aggressive_mode=True
    )

if __name__ == "__main__":
    result = main()